In [ ]:
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob
import platform
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Paths Input Here
analysis = "rios_worst_responder_top_responder_nonoverlap_map"
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    image_one = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/rios_2022_networks/overlap_map/rios_worst_responders_conjunction__map_derived_fromtop-vs-worst_specificity/rios_worst_responders_conjunction__map_derived_fromtop-vs-worst_specificity_stdevthresh_0_voxthresh_None_absval_True_positive_and_negative_values_raw_map.nii'    
    image_two = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/rios_2022_networks/overlap_map/rios_top_responders_conjunction__map_derived_fromtop-vs-worst_specificity/rios_top_responders_conjunction__map_derived_fromtop-vs-worst_specificity_stdevthresh_0_voxthresh_None_absval_True_positive_and_negative_values_raw_map.nii'
    out_dir = rf'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/rios_2022_networks/nonoverlap_maps/{analysis}'
    #out_dir = r'path to out dir here'
    print('I have set pathnames in the Mac style')
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\stats'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')

In [ ]:
#Load and prepare images
image_one = image.load_img(image_one)
#Organize files into manipulatable dataframes
data1 = image_one.get_fdata(); data1 = np.nan_to_num(data1, nan=0, posinf=0, neginf=0)

image_two = image.load_img(image_two)
data2 = image_two.get_fdata(); data2 = np.nan_to_num(data2, nan=0, posinf=0, neginf=0)

print('Number of mismatching voxels: ', np.sum(data1!=data2))

In [ ]:
# For Top Response Condition

#generate a matrix that finds all regions of matching amplitude and sign
mask_matrix = np.ones(data1.shape)
mask_matrix[data1 == data2] = 0

#set all regions meeting above match criteria to zero in each map. 
data1_nonoverlap = data1*mask_matrix
data2_nonoverlap = data2*mask_matrix

print('Number of mismatching voxels: ', np.sum(data1_nonoverlap != data2_nonoverlap))
print('Number voxels map 1: ', np.sum(np.abs(data1_nonoverlap)))
print('Number voxels map 2: ', np.sum(np.abs(data2_nonoverlap)))

In [ ]:
#generate a nifti from the top response specificity map
mask = nimds.get_img("mni_icbm152")
ovr_img1 = image.new_img_like(mask, data1_nonoverlap)
ovr_html1 = plotting.view_img(ovr_img1, cut_coords=(0,0,0), black_bg=False, opacity=.75, cmap='ocean_hot')
ovr_html1

In [ ]:
#generate a nifti from the bottom response specificity map
ovr_img2 = image.new_img_like(mask, data2_nonoverlap)
ovr_html2 = plotting.view_img(ovr_img2, cut_coords=(0,0,0), black_bg=False, opacity=.75, cmap='ocean_hot')
ovr_html2

In [ ]:
#Save
if os.path.isdir(out_dir)==False:
    os.makedirs(out_dir)

#Save
ovr_img1.to_filename(os.path.join(out_dir, f'{analysis}_impair_nonoverlap'))
ovr_html1.save_as_html(os.path.join(out_dir, f'{analysis}_impair_nonoverlap.html'))
ovr_img2.to_filename(os.path.join(out_dir, f'{analysis}_improve_nonoverlap'))
ovr_html2.save_as_html(os.path.join(out_dir, f'{analysis}_improve_nonoverlap.html'))



print('Title: ' + 'nonoverlap maps')
print('saved to: ', out_dir)